In [1]:
import pandas as pd
import glob, os

datadir = os.path.join("data","opendata","SI","car_registrations")

In [2]:
dt = pd.DataFrame()
for i in glob.glob(datadir + os.sep + "*.csv"):
    dt = pd.concat((dt, pd.read_csv(i, sep=";", encoding="ISO8859-16")))

C:\PROGLANG\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (44,45,82) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\PROGLANG\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (15,17,28,41,50,91) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\PROGLANG\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (28,44,54,62,90,91) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\PROGLANG\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (28,44,45,62,91,98) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, com

In [3]:
dt["status_dt"]  = pd.to_datetime(dt["Datum statusa vozila"])

In [4]:
dt.shape

(109173, 105)

In [5]:
pd.options.display.max_columns = 500
dt.head()

,1K-Oznaka potrdila o skladnosti,2A-Datum prve registracije vozila v SLO,4A-Podvrsta tablice prve registracije,4A-Registrsko obmocje tablice prve registracije,5A-Leto izdelave,8B-Namen vozila,B-Datum prve registracije vozila,C-Ali je uporabnik pravna ali fizicna oseba,C-Ali je uporabnik tudi lastnik vozila,C-Spol uporabnika (ce gre za fizicno osebo),C-Starost uporabnika vozila,C1.3-Obcina uporabnika vozila (opis),C1.3-Obcina uporabnika vozila (oznaka),C1.3-Upravna enota uporabnika vozila (opis),C1.3-Upravna enota uporabnika vozila (oznaka),C2-Ali je lastnik pravna ali fizicna oseba,C2-Spol lastnika (ce gre za fizicno osebo),C2-Starost lastnika vozila,D.1-Znamka,D.2-Tovar. oznaka,D.3-Komerc. oznaka,D.4.2-Drzava (koda),D.4.2-Drzava (opis),D.5-Datum izdaje COC dokumenta oz. prve registracije,Datum izdaje potrdila o skladnosti,Datum statusa vozila,E-VIN,E.1-Identifikacijska koda,F. 2-Najvecja dovoljena masa vozila pri registraciji,F.1-Najvecja tehnicno dovoljena masa vozila,F.2-Najvecja dovoljena masa vozila pri registraciji,F.3-Najvecja dovoljena masa skupine vozil pri registraciji,G-Masa vozila,Izvajalna enota prve registracije,J-Kategorija in vrsta vozila (opis),J-Kategorija in vrsta vozila (oznaka),K-Homologacijska oznaka vozila,Komerc. oznaka do prvega /,L-Stevilo osi,M-Medosje,M.1-Zadnji previs,N-Razporeditev najv. tehnicno dovoljene mase na osi,N.1-Dovoljene osne obremenitve,O.1.1-Priklopnik,O.1.2-Polpriklopnik,O.1.3-Priklopnik s centralno osjo,O.2-Nezavirano priklopno vozilo,O.3-Navpicna obremenitev vlecne naprave / sedla,O.4-Tlak v napajalnem vodu zavornega sistema,Okoljevarstvena oz naka,Okoljevarstvena oznaka,P.1.1-Delovna prostornina,P.1.2-Nazivna moc,P.1.3-Vrsta goriva (opis),P.1.3-Vrsta goriva (oznaka),P.1.4-Nazivna vrtilna frekvenca,P.1.5-Oznaka motorja,P.2.1-Tip,P.2.2-Nazivna trajna moc,P.2.3-Delovna napetost,P.2.4-Pogonske baterije,P.2.5-Oznaka motorja,Pogonska os z zracnim vzmetenjem ali priznanim ekvivalentom,Q-Razmerje moc/masa (samo za motorna kolesa),R-Barva vozila (opis),R-Barva vozila (oznaka),S.1-Stevilo sedezev (vkljucno z vozniskim ),S.1-Stevilo sedezev (vkljucno z vozniskim),S.2-Stevilo stojisc,Status vozila (id),Status vozila (opis),T-Najvisja hitrost,U.1-V mirovanju,U.2-Pri vrtilni frekvenci motorja,U.3-V voznji,V.1-CO,"V.10-Vrtilna frekvenca prostega teka, nizka",V.10.1-Vsebina CO,V.10.2-Temperatura olja,"V.11-Vrtilna frekvenca prostega teka, visoka",V.11.1-Vsebina CO,V.11.2-Vrednost Lambda,V.11.3-Temperatura olja,V.2-HC,V.3-Nox,V.4-HC + Nox,V.5-Delci pri dizel motorjih,V.6-Korigiran absorpcijski koeficient pri dizel motorjih,V.6.1-Vrtilna frekvenca prostega teka,V.6.2-Najvisja vrtilna frekvenca motorja,V.6.3-Temperatura olja,V.7-CO2,V.8-Kombinirana poraba goriva,V.9-Podatek o okoljevarstveni kategoriji vozila,X-Oblika nadgradnje (opis),X-Oblika nadgradnje (oznaka),X.1-Dodatni opis nadgradnje,X.2-Homologacijska oznaka zascitne konstrukcije,Y.1-Dolzina,Y.2-Sirina,Y.3-Visina,Z.1-Dovoljene pnevmatike in platisca,Z.2-Homologacijska oznaka vlecne naprave,Z.2.1-D Đ vrednost,status_dt
0,SB,05.01.2017,NT,KRANJ,2010.0,NaN,23.04.2010,F,DA,Z,44.0,Cerklje na Gorenjskem,12,KRANJ,18,F,Z,44,AUDI,ACAYCF1-FM5A4051R8P617MGEM1,A3 / 1.6 / TDI,DEU,Nemèija,23.04.2010,07.12.2016,05.01.2017,WAUZZZ8P7AA152434,NaN,NaN,1840,NaN,3290.0,1355,"PRODUKT & CO., tehnièni pregledi d.o.o.",osebni avtomobil,M1,e1*2001/116*0217*27,A3,2.0,2578,"779,0",NaN,1015/950,NaN,NaN,"1400,0",670.0,75,NaN,NaN,EURO 5,1598,77,Dizel,D,4400,CAY,NaN,NaN,NaN,NaN,NaN,NaN,NaN,navaden - RDEÈA - SREDNJA,S3M,NaN,5.0,NaN,1,registrirano,194,72,2500.0,72,"0,2528",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"0,1582","0,1834","0,0004","0,5",NaN,NaN,NaN,109,"4,1",715/2007*692/2008A (EURO 5a),karavan,AC,-,NaN,4238.0,1765,1406,"205/50 R17 93H M+S, 205/55 R16 91V, 205/55 R16...",NaN,NaN,2017-05-01
1,SA,31.01.2017,NT,CELJE,2017.0,NaN,31.01.2017,F,NE,Z,61.0,Radeèe,99,LAKO,20,F,M,55,RENAULT,5RSN0A,CLIO / 1.2 / 16V,FRA,Francija,04.01.2017,04.01.2017,31.01.2017,VF15RSN0A56425683,NaN,Na

In [16]:
dt.rename(columns = {"4A-Registrsko obmocje tablice prve registracije":"region"}, inplace=True)

In [17]:
tmp = dt["region"].value_counts()

In [18]:
tmp

LJUBLJANA         38135
MARIBOR           14862
CELJE             14264
KOPER              8813
KRANJ              8434
NOVO MESTO         6428
NOVA GORICA        5485
MURSKA SOBOTA      5142
SLOVENJ GRADEC     3086
KRKO              2686
POSTOJNA           1440
Policija            345
CD                   32
M                    15
CC                    3
CMD                   2
Neznano               1
Name: region, dtype: int64

In [19]:
import numpy as np
from bokeh.io import show, output_notebook
from bokeh.plotting import figure

output_notebook()
TOOLS = "hover"
p = figure(y_range = list(tmp.index.values[::-1]), plot_height = 300, title = "Registrsko Obmocje", tools = TOOLS)

p.hbar(y = tmp.index.values[::-1], right = tmp.values[::-1], height  =1)
p.ygrid.grid_line_color = None
p.x_range.start = 0

show(p)

Loading BokehJS ...

In [20]:
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.palettes import Category20
from bokeh.plotting import figure
from bokeh.transform import factor_cmap


output_notebook()
TOOLS = "hover"
regions =  list(tmp.index.values[::-1])
vals  = list(tmp.values[::-1])

src = ColumnDataSource(data = dict(regions = regions, vals = vals))
p = figure(y_range = list(tmp.index.values[::-1]), plot_height = 300, plot_width = 800, title = "Registrsko Obmocje", tools = TOOLS)

p.hbar(y = "regions" , right = "vals", source = src, height  =1, 
       line_color="white", fill_color = factor_cmap('regions', palette = Category20[20], factors =  regions))
p.ygrid.grid_line_color = None
p.x_range.start = 0
p.legend.orientation = "vertical"
p.legend.location = "bottom_right"


show(p)

Loading BokehJS ...

# selecting top 5 regions by month

In [10]:
import calendar
dt["reg_month"] = dt["status_dt"].dt.month.apply(lambda x: calendar.month_abbr[x])

In [11]:
top5_names = tmp.index.values[:5]

In [21]:
gr = (dt.loc[dt["region"].isin(top5_names),:]
               .groupby([ "reg_month", "region" ]))

In [27]:
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource, FactorRange
from bokeh.plotting import figure


output_notebook()
TOOLS = "hover"

src = ColumnDataSource(data = gr)

p = figure(y_range = gr, plot_height = 300, plot_width = 800, 
           title = "Region of registration by month", tools = TOOLS, toolbar_location = None)

p.hbar(y =  ("reg_month", "region" ) , right = "region_min", source = src, height  =1, 
       line_color="white")
p.ygrid.grid_line_color = None
p.x_range.start = 0
p.legend.orientation = "vertical"
p.legend.location = "bottom_right"


show(p)

Loading BokehJS ...

C:\PROGLANG\Anaconda3\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
C:\PROGLANG\Anaconda3\lib\site-packages\bokeh\models\sources.py:137: BokehUserWarning: ColumnDataSource's columns must be of the same length. Current lengths: ('5A-Leto izdelave_25%', 60), ('5A-Leto izdelave_50%', 60), ('5A-Leto izdelave_75%', 60), ('5A-Leto izdelave_count', 60), ('5A-Leto izdelave_max', 60), ('5A-Leto izdelave_mean', 60), ('5A-Leto izdelave_min', 60), ('5A-Leto izdelave_std', 60), ('C-Starost uporabnika vozila_25%', 60), ('C-Starost uporabnika vozila_50%', 60), ('C-Starost uporabnika vozila_75%', 60), ('C-Starost uporabnika vozila_count', 60), ('C-Starost uporabnika vozila_max', 60), ('C-Starost uporabnika vozila_mean', 60), ('C-Starost uporabnika vozila_min', 60), ('C-Starost uporabnika vozila_std', 